In [17]:
using Revise
using GLMakie
include("EFITModule/EFIT.jl")
using NPZ

In [18]:
using ParallelStencil
using ParallelStencil.FiniteDifferences3D
using GLMakie

ParallelStencil.@reset_parallel_stencil()
USE_GPU = false
@static if USE_GPU
    @init_parallel_stencil(CUDA, Float32, 3)
else
    @init_parallel_stencil(Threads, Float32, 3)
end

In [23]:
matGrid = npzread("MeshFiles/IIWMini.npy")[:,:,1:90].+1


#Maximum sound speed in the model
cMax = 8000
cMin = 6000
#Frequency, hz
f0=2.25e6
#Period
t0 = 1.00 / f0
#Maximum spatial increment
dx = cMin/(8*f0)
#Maximum time increment
dt = dx/(cMax*sqrt(3))



Creating grid of size 167, 497, 90

0.0


2.4056261216234407e-8

In [37]:
display(volume(matGrid.-1))

GLMakie.Screen(...)

In [25]:
materials = [Main.EFIT.IsoMat(2,1,1), Main.EFIT.IsoMats["AISI1018"]]

2-element Vector{Main.EFIT.IsoMat}:
 Main.EFIT.IsoMat(1.0f0, 2.0f0, 1.0f0, 4.0f0)
 Main.EFIT.IsoMat(7870.0f0, 1.11923f11, 8.1852916f10, 2.7562885f11)

In [39]:
grid = Main.EFIT.EFITGrid(Int16.(matGrid),materials,dt,dx);

sourcePositions = [(160:165,200:240,35:55)]

#Create a source function from which input can be taken
function source(t)
    v = exp(-((2*(t-2*t0)/(t0))^2))*sin(2*pi*f0*t)*0.1
    return Data.Number(v)
end
nframes = 600
framerate = 30

tIterator = 0:grid.dt:grid.dt*nframes
fig,ax,plt = volume(Array(grid.vx),algorithm=:mip,colorrange = (0, 0.1),colormap=:curl, transparency=true)
#Step the simulation
function stepSim(t)
    println(t)
    #Isotropic step function
    Main.EFIT.SimStep!(grid)
    #Apply the source to the Z-direction
    @parallel (sourcePositions[1]) Main.EFIT.applySource!(grid.vx,grid.vy,grid.vz, Data.Number(0.0), Data.Number(0.0), (source(t)))
    velMag = sqrt.(grid.vx.^2 .+ grid.vy.^2 .+ grid.vz.^2)
    #Update the plot
    plt.volume = velMag
end

record(stepSim, fig, "color_animation.mp4", tIterator; framerate = 30)


Creating grid of size 167, 497, 90

0.0

Creating grid of size 167, 497, 90

0.0
2.4056261e-8
4.8112522e-8
7.216878e-8
9.6225044e-8
1.202813e-7
1.4433756e-7
1.6839383e-7
1.9245009e-7
2.1650635e-7
2.405626e-7
2.6461888e-7
2.8867512e-7
3.127314e-7
3.3678765e-7
3.6084393e-7
3.8490018e-7
4.0895642e-7
4.330127e-7
4.5706895e-7
4.811252e-7
5.051815e-7
5.2923775e-7
5.53294e-7
5.7735025e-7
6.014065e-7
6.254628e-7
6.4951905e-7
6.735753e-7
6.9763155e-7
7.2168785e-7
7.457441e-7
7.6980035e-7
7.938566e-7
8.1791285e-7
8.4196915e-7
8.660254e-7
8.9008165e-7
9.141379e-7
9.381942e-7
9.622504e-7
9.863066e-7
1.010363e-6
1.0344193e-6
1.0584755e-6
1.0825318e-6
1.106588e-6
1.1306443e-6
1.1547005e-6
1.1787567e-6
1.202813e-6
1.2268694e-6
1.2509256e-6
1.2749819e-6
1.2990381e-6
1.3230944e-6
1.3471506e-6
1.3712068e-6
1.3952631e-6
1.4193193e-6
1.4433757e-6
1.467432e-6
1.4914882e-6
1.5155445e-6
1.5396007e-6
1.563657e-6
1.5877132e-6
1.6117694e-6
1.6358257e-6
1.6598821e-6
1.6839383e-6
1.7079946e-6
1.7320508e-6
1.7561

"color_animation.mp4"